In [1]:
import os
# os.environ["CUDA_VISIBLE_DEVICES"]="0"
os.environ["FLASK_DEBUG"]="1"

In [2]:
!pip install numpy
!pip install flask
!pip install flask_restful
!pip install pandas
!pip install tensorflow
!pip install tqdm
!pip install sklearn
!pip install torch==1.7.0+cpu torchvision==0.8.1+cpu torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html
!pip install laserembeddings
!pip install sentence_transformers
!python -m laserembeddings download-models

Looking in links: https://download.pytorch.org/whl/torch_stable.html



   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

 You're all set!


In [3]:
import logging
import os
import typing
import warnings
from typing import Any, Dict, List, Optional, Text, Tuple, Type
import pickle
import numpy as np
####


from datetime import datetime
import csv
import json
import abc
import pandas as pd
# import glog
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from IPython.display import display
from laserembeddings import Laser
from sentence_transformers import SentenceTransformer
from sklearn.metrics import accuracy_score

_TRAIN = "train"
_TEST = "test"
_VAL = 'validation'
batchsize_super = 60

In [4]:
# from abc import ABCMeta, abstractmethod

In [5]:
import collections
# from typing import Dict, Optional

# import numpy as np

# _MAX_PER_BATCH = 3

In [6]:
#######

logger = logging.getLogger(__name__)

if typing.TYPE_CHECKING:
    import sklearn

In [7]:
class SamplingBatcher(collections.abc.Iterator):
    """Batcher that samples according to a given distribution.

    It defaults to sampling from the data distribution.

    WARNING: this class is not deterministic. if you want deterministic
    behaviour, just freeze the numpy seed.
    """

    def __init__(
            self,
            examples: np.ndarray,
            labels: np.ndarray,
            batch_size: int,
            sample_distribution: Optional[Dict[int, float]] = None,
    ):
        """Create a new BalancedBatcher.

        Args:
            examples: np.ndarray containing examples
            labels: np.ndarray containing labels
            batch_size: int size of a single batch
            sample_distribution: optional distribution over label
                classes for sampling. This is normalized to sum to 1. Defines
                the target distribution that batches will be sampled with.
                Defaults to the data distribution.
        """
        _validate_labels_examples(examples, labels)
        self._examples = examples
        self._labels = labels
        self._label_classes = np.unique(labels)
        self._class_to_indices = {
            label: np.argwhere(labels == label).flatten()
            for label in self._label_classes
        }
        if sample_distribution is None:
            # Default to the data distribution
            sample_distribution = {
                label: float(indices.size)
                for label, indices in self._class_to_indices.items()
            }
        self._label_choices, self._label_probs = (
            self._get_label_choices_and_probs(sample_distribution))
        self._batch_size = batch_size

    def _get_label_choices_and_probs(self, sample_distribution):
        label_choices = []
        label_probs = []
        weight_sum = sum(sample_distribution.values())
        for label, weight in sample_distribution.items():
            if label not in self._labels:
                raise ValueError(
                    f"label {label} in sample distribution does not exist")
            if weight < 0.0:
                raise ValueError(
                    f"weight {weight} for label {label} is negative")
            label_choices.append(label)
            label_probs.append(weight / weight_sum)

        return np.array(label_choices), np.array(label_probs)

    def __next__(self):
        """Generates the next batch.

        Returns:
            (batch_of_examples, batch_of_labels) - a tuple of ndarrays
        """
        class_choices = np.random.choice(
            self._label_choices, size=self._batch_size, p=self._label_probs)

        batch_indices = []
        for class_choice in class_choices:
            indices = self._class_to_indices[class_choice]
            batch_indices.append(np.random.choice(indices))

        return self._examples[batch_indices], self._labels[batch_indices]

    def __iter__(self):
        """Gets an iterator for this iterable

        Returns:
            self because the class is an iterator itself
        """
        return self

In [8]:
class ClassificationEncoderClient(object):
    """A model that maps from text to dense vectors."""
    __metaclass__ = abc.ABCMeta

    @abc.abstractmethod
    def encode_sentences(self, sentences):
        """Encodes a list of sentences

        Args:
            sentences: a list of strings

        Returns:
            an (N, d) numpy matrix of sentence encodings.
        """
        return NotImplementedError


In [9]:
class CombinedEncoderClient(ClassificationEncoderClient):
    """concatenates the encodings of several ClassificationEncoderClients

    Args:
        encoders: A list of ClassificationEncoderClients
    """

    def __init__(self, encoders: list):
        """constructor"""
        self._encoders = encoders

    def encode_sentences(self, sentences):
        """Encode a list of sentences

        Args:
            sentences: the list of sentences

        Returns:
            an array with shape (len(sentences), ENCODING_SIZE)
        """
        encodings = np.hstack([encoder.encode_sentences(sentences)
                               for encoder in self._encoders])
        print('DEBUG combined size:', encodings.shape)
        return encodings



class LaserEncoderClient(ClassificationEncoderClient):
    """A wrapper around ClassificationEncoderClient to normalise the output"""

    def __init__(self, batch_size=100):
        """Create a new ConvertEncoderClient object

        Args:
            uri: The uri to the tensorflow_hub module
            batch_size: maximum number of sentences to encode at once
        """
        self._batch_size = batch_size
        self._encoder_client = Laser()

    def encode_sentences(self, sentences):
        """Encode a list of sentences

        Args:
            sentences: the list of sentences

        Returns:
            an (N, d) numpy matrix of sentence encodings.
        """
        encodings = []
        #         glog.setLevel("ERROR")
        for i in tqdm(range(0, len(sentences), self._batch_size),
                      "encoding sentence batches"):
            encodings.append(
                self._encoder_client.embed_sentences(
                    sentences[i:i + self._batch_size], lang='en'))
        #         glog.setLevel("INFO")
        print('DEBUG LASER SIZE:', np.vstack(encodings).shape)
        return l2_normalize(np.vstack(encodings))


class SbertEncoderClient(ClassificationEncoderClient):
    """A wrapper around ClassificationEncoderClient to normalise the output"""

    def __init__(self, sbert_model, batch_size=batchsize_super):
        """Create a new ConvertEncoderClient object

        Args:
            uri: The uri to the tensorflow_hub module
            batch_size: maximum number of sentences to encode at once
        """
        self._batch_size = batch_size
        self._encoder_client = SentenceTransformer(sbert_model)

    def encode_sentences(self, sentences):
        """Encode a list of sentences

        Args:
            sentences: the list of sentences

        Returns:
            an (N, d) numpy matrix of sentence encodings.
        """
        encodings = []
        #         glog.setLevel("ERROR")
        for i in tqdm(range(0, len(sentences), self._batch_size),
                      "encoding sentence batches"):
            encodings.append(
                self._encoder_client.encode(
                    sentences[i:i + self._batch_size]))
        #         glog.setLevel("INFO")
        return l2_normalize(np.vstack(encodings))
def l2_normalize(encodings):
    """L2 normalizes the given matrix of encodings."""
    norms = np.linalg.norm(encodings, ord=2, axis=-1, keepdims=True)
    return encodings / norms


class PolynomialDecay:
    """A callable that implements polynomial decay.

    Used as a callback in keras.
    """

    def __init__(self, max_epochs, init_lr, power=1.0):
        """Creates a new PolynomialDecay

        Args:
            max_epochs: int, maximum number of epochs
            init_lr: float, initial learning rate which will decay
            power: float, the power of the decay function
        """
        self.max_epochs = max_epochs
        self.init_lr = init_lr
        self.power = power

    def __call__(self, epoch):
        """Calculates the new (smaller) learning rate for the current epoch

        Args:
            epoch: int, the epoch for which we need to calculate the LR

        Returns:
            float, the new learning rate
        """
        decay = (1 - (epoch / float(self.max_epochs))) ** self.power
        alpha = self.init_lr * decay

        return float(alpha)


def _create_model(hparams):
    model = tf.keras.models.Sequential()
    dropout = hparams.dropout
    optimizer_name = hparams.optimizer
    optimizer = {
        'adam': tf.keras.optimizers.Adam,
        'sgd': tf.keras.optimizers.SGD
    }[optimizer_name]

    input_size = hparams.input_size
    for _ in range(hparams.num_hidden_layers):
        model.add(
            tf.keras.layers.Dropout(dropout, input_shape=(input_size,))
        )
        model.add(tf.keras.layers.Dense(hparams.hidden_layer_size,
                                        activation=hparams.activation))
        input_size = hparams.hidden_layer_size

    model.add(tf.keras.layers.Dense(units=1))

    model.compile(loss="mean_absolute_error",
                  optimizer=optimizer(lr=hparams.learning_rate),
                  metrics=["accuracy"])
    return model



def _validate_labels_examples(examples, labels):
    if not isinstance(examples, np.ndarray):
        raise ValueError("examples should be an ndarray")

    if not isinstance(labels, np.ndarray):
        raise ValueError("labels should be ndarray")

    if not labels.size == examples.shape[0]:
        raise ValueError("number of labels != number of examples")


def iter_to_generator(iterator):
    """Gets a generator from an iterator.

    Used so that keras type checking does not complain.

    Args:
        iterator: any python iterator

    Returns:
        a python generator that just calls next on the iterator
    """

    def gen():
        while True:
            yield next(iterator)

    return gen()


def train_model(train_encodings, train_labels, categories, hparams,
                validation_data=None, verbose=1):
    """Trains an intent classification model

    Args:
        train_encodings: np.array with the train encodings
        train_labels: list of labels corresponding to each train example
        categories: the set of categories
        hparams: a tf.contrib.training.HParams object containing the model
            and training hyperparameters
        validation_data: (validation_encodings, validation_labels) tuple
        verbose: the keras_model.train() verbose level

    Returns:
        model: a keras model
        eval_acc_history: The evaluation results per epoch

    """
    distribution = None if not hparams.balance_data else {
        x: 1. / len(categories) for x in range(len(categories))}

    batcher = SamplingBatcher(
        train_encodings, train_labels, hparams.batch_size, distribution)

    steps_per_epoch = np.ceil(len(train_labels) / hparams.batch_size)

    model, eval_acc_history = _train_mlp_with_generator(
        batcher, train_encodings.shape[1], steps_per_epoch,
        categories, hparams, validation_data=validation_data, verbose=verbose)
    return model, eval_acc_history


def _train_mlp_with_generator(
        batcher, input_size, steps_per_epoch, label_set, hparams,
        validation_data=None, verbose=1):
    """Trains a Multi Layer Perceptron (MLP) model using keras.

    Args:
        batcher: an instance of a class that inherits from abc.Iterator and
            iterates through batches. see batchers.py for an example.
        input_size: int, length of the input vector
        steps_per_epoch: int, number of steps per one epoch
        label_set: set of ints, the set of labels
        hparams: an instance of tf.contrib.training.Hparams, see config.py
            for some examples
        validation_data: This can be either
            - a generator for the validation data
            - a tuple (inputs, targets)
            - a tuple (inputs, targets, sample_weights).
        verbose: keras verbosity mode, 0, 1, or 2.

    Returns:
        keras model, which has been trained
        test accuracy history, as retreived from keras
    """

    hparams.input_size = input_size
    hparams.output_size = len(label_set)
    #     print('!!!! validation data',validation_data)
    model = _create_model(hparams)

    callbacks = None
    if hparams.lr_decay_pow:
        callbacks = [
            tf.keras.callbacks.LearningRateScheduler(PolynomialDecay(
                max_epochs=hparams.epochs,
                init_lr=hparams.learning_rate,
                power=hparams.lr_decay_pow))]

    #     glog.info("Training model...")
    history_callback = model.fit_generator(
        generator=iter_to_generator(batcher),
        steps_per_epoch=max(steps_per_epoch, 1),
        epochs=hparams.epochs,
        shuffle=False,
        validation_data=validation_data,
        callbacks=callbacks,
        verbose=verbose
    )

    test_acc_history = (None if not validation_data
                        else history_callback.history["val_accuracy"])

    return model, test_acc_history


class hparamset():
    def __init__(self):
        self.batchsize = 32
        self.balance_data = False
        self.output_size = None
        self.activation = 'relu'
        self.hidden_layer_size = 512
        self.num_hidden_layers = 1
        self.batch_size = 32
        self.dropout = 0.75
        self.optimizer = 'sgd'
        self.learning_rate = 0.7
        self.lr_decay_pow = 1
        self.epochs = 100
        self.eval_each_epoch = True

    ###############


#############



In [10]:
sbert_model = 'distiluse-base-multilingual-cased'
sbert_model2 = 'xlm-r-100langs-bert-base-nli-stsb-mean-tokens'
sbert_model3 = 'distilbert-multilingual-nli-stsb-quora-ranking'
sbert_encoder = SbertEncoderClient(sbert_model)
sbert_encoder2 = SbertEncoderClient(sbert_model2)
sbert_encoder3 = SbertEncoderClient(sbert_model3)
laser_encoder = LaserEncoderClient()
encoder_client = CombinedEncoderClient([laser_encoder, sbert_encoder, sbert_encoder2, sbert_encoder3])

In [11]:
# pickle.dump(encoder_client, open('encoders_pickle.pkl','wb') )
# encoder_client = pickle.load(open('encoders_pickle.pkl','rb') )

In [12]:
from flask import Flask
from flask_restful import Resource, Api, reqparse

In [13]:
app = Flask(__name__)
api = Api(app)

In [14]:
# global model
# model = None

In [15]:
model = None
class FlaskServingModel(Resource):
    def __init__(self):
        self._required_features = ['text', 'labels', 'unique_labels']
        self.reqparse = reqparse.RequestParser()
        for feature in self._required_features:
            self.reqparse.add_argument(
                feature, type = list, required = True, location = 'json',
                help = 'No {} provided'.format(feature))
        self.hparams = hparamset()
        self.verbose = 0
        self.encodings = {}
        self.X = []
        self.y = []
        self.training_text  = []
        self.categories = {}
        self.eval_acc_history = None
        self.test_text = None
        print('INITIALISED')
        super(FlaskServingModel, self).__init__()
        
    def put(self):
        """
        TRAIN METHOD
        """
        args = self.reqparse.parse_args()        
        self.training_text = args['text']
        self.y = np.array(args['labels'])
        self.categories = args['unique_labels']
        
        enc = encoder_client.encode_sentences(self.training_text)#train.text.values[:100])#
        global model
        model, eval_acc_history = train_model(enc, self.y, self.categories, self.hparams, validation_data=None,verbose=0)
        model.save('trained_model.tf')
        print('model trained AND saved')
        return {'status':'trained and saved'}   
    def post(self):
        """
        PREDICTION METHOD
        """
        args = self.reqparse.parse_args()        
        enc = encoder_client.encode_sentences(args['text']) 
        print(f'inferring on: {args["text"]}')
        global model
        pred = model.predict(enc) #enc[:100])
        print(pred)
        return {'prediction':pred.tolist()}
    
    def get(self):
        """
        EVALUATE
        """
        args = self.reqparse.parse_args()
        self.test_text = encoder_client.encode_sentences(args['text']) 
        global model
        out = model.predict(self.test_text)
        prediction = np.argmax(out, axis=1)
        s= pd.DataFrame({'pred':prediction,'testlabels':args['labels']})#, 'language':test.language})#.to_csv('xlni_preds_temp.csv') #lang
        ax_test_acc = accuracy_score(s.testlabels, s.pred)
        return {'accuracy':ax_test_acc}

In [16]:
# mod = FlaskServingModel()

In [17]:
# test = pd.read_csv('/polyai-models/data/polyai-models/200909_test_huawei_wallet_30strat_min3_manyanswers.csv')
# train = pd.read_csv('/polyai-models/data/polyai-models/200909_train_huawei_wallet_60strat_min3_manyanswers.csv')
# val = pd.read_csv('/polyai-models/data/polyai-models/200909_val_huawei_wallet_10strat_min3_manyanswers.csv')

In [18]:
# mod.put()

In [19]:
# mod.post()

In [20]:
api.add_resource(FlaskServingModel, '/train', '/predict', '/evaluate')
# api.add_resource(FlaskServingModel, '/train')
# api.add_resource(FlaskServingModel, '/predict')

In [ ]:
if __name__ == '__main__':
    app.run(debug=True,  port=9501, use_reloader=False)#host='0.0.0.0',

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


INFO:werkzeug: * Running on http://127.0.0.1:9501/ (Press CTRL+C to quit)
encoding sentence batches:   0%|                                                                                                 | 0/1 [00:00<?, ?it/s]

INITIALISED


encoding sentence batches:   0%|                                                                                                 | 0/2 [00:00<?, ?it/s]

DEBUG LASER SIZE: (89, 1024)


...

INITIALISED
DEBUG LASER SIZE: (12, 1024)



INFO:werkzeug:127.0.0.1 - - [27/Nov/2020 23:52:55] "POST /predict HTTP/1.1" 200 -


DEBUG combined size: (12, 3072)
inferring on: ['H', 'o', 'w', ' ', 'a', 'r', 'e', ' ', 'y', 'o', 'u', '?']
[[2.1213613e-01 7.8689121e-03 8.1633320e-03 1.5856759e-01 8.8944390e-02
  4.8694378e-01 6.5328395e-03 3.0842969e-02]
 [5.8755934e-01 2.8886942e-03 3.2760878e-03 1.4252669e-01 3.8362060e-02
  1.9087939e-01 2.4856059e-03 3.2022119e-02]
 [2.2774069e-01 7.1952697e-03 6.7250994e-03 2.3231927e-01 1.8612449e-01
  2.7083510e-01 6.4712805e-03 6.2588781e-02]
 [5.7118630e-01 3.8775357e-03 4.1034226e-03 1.7053838e-01 5.8352832e-02
  5.3972613e-02 4.0568849e-03 1.3391191e-01]
 [6.2336302e-01 2.9429777e-03 3.4515695e-03 1.6951038e-01 8.9116074e-02
  7.5608701e-02 3.3230446e-03 3.2684226e-02]
 [2.7584463e-01 9.6707093e-03 9.6074874e-03 2.4118851e-01 1.6060558e-01
  2.2172132e-01 1.4144824e-02 6.7216963e-02]
 [8.7138617e-01 9.7324705e-04 9.5790881e-04 6.4093664e-02 1.7214231e-02
  2.5646145e-02 2.1858176e-03 1.7542768e-02]
 [5.7118630e-01 3.8775357e-03 4.1034226e-03 1.7053838e-01 5.8352832e-02
  

encoding sentence batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 32.26it/s]
INFO:werkzeug:127.0.0.1 - - [27/Nov/2020 23:53:00] "POST /predict HTTP/1.1" 200 -


INITIALISED
DEBUG LASER SIZE: (7, 1024)
DEBUG combined size: (7, 3072)
inferring on: ['B', 'o', 'n', 'j', 'o', 'u', 'r']
[[3.75780731e-01 1.05415285e-02 9.55444388e-03 2.39334702e-01
  1.46029800e-01 1.63915664e-01 6.48473995e-03 4.83583957e-02]
 [5.87559342e-01 2.88869417e-03 3.27608781e-03 1.42526686e-01
  3.83620597e-02 1.90879390e-01 2.48560589e-03 3.20221186e-02]
 [2.94192526e-02 1.23094989e-03 1.58568402e-03 9.32311177e-01
  7.19654793e-03 1.54104978e-02 2.03080999e-04 1.26428325e-02]
 [2.14024559e-01 6.88460842e-03 9.73494444e-03 1.87073007e-01
  1.49131611e-01 4.02556121e-01 3.85217112e-03 2.67430339e-02]
 [5.87559342e-01 2.88869417e-03 3.27608781e-03 1.42526686e-01
  3.83620597e-02 1.90879390e-01 2.48560589e-03 3.20221186e-02]
 [1.89547360e-01 5.24676358e-03 7.02723907e-03 1.67397931e-01
  1.60890400e-01 4.20428365e-01 2.76052789e-03 4.67013121e-02]
 [2.75844663e-01 9.67071019e-03 9.60748922e-03 2.41188526e-01
  1.60605595e-01 2.21721277e-01 1.41448248e-02 6.72169700e-02]]


In [ ]:
# enc = encoder_client.encode_sentences(train.text.values[:100])

In [ ]:
# model, eval_acc_history = train_model(enc, train.labels.values[:100], categories=train.labels.unique(), hparams=hparamset(), validation_data=None,verbose=1)

In [ ]:
# model.predict(enc[:100])

In [ ]:
# model.save('test.tr')

In [ ]:
# model=load_model('test.tr')
# model = load_model('trained_model.tf')

In [ ]:
# model.predict(enc[:100]).shape

In [ ]:
# mod.post({'text':train.text.values, 'labels':train.labels.values})

In [ ]:
# # args = self.reqparse.parse_args()
# enc = encoder_client.encode_sentences(train.text.values[:100]) 
# # print(f'inferring on: {args["text"]}')
# model = load_model('trained_model.tf')
# print('MODEL LOADED')
# pred = model.predict(enc)
# print(pred)
# # return {'prediction':pred.tolist()}